In [1]:
import os
import bs4
import regex
import shutil
import zipfile
from bs4 import BeautifulSoup

In [2]:
# from txt_to_dataset import find_characters
# from txt_to_dataset import find_author
# from txt_to_dataset import find_end

---

In [3]:
def printlines(lines):
    print('\n'.join(lines))

In [54]:
def printsep(s):
    print(s*40)

In [18]:
def underprint(x):
    print(x)
    print('-'*len(x))

In [5]:
def get_lines(txt_dir, fname):
    with open(os.path.join(txt_dir, fname), "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()
        lines = raw.split('\n')
    return raw, lines, len(lines)    

In [192]:
def print_author_split(lines, index):
    printlines(lines[:index])
    printsep('*')
    printlines(lines[index:index+10])

In [189]:
def print_file(f):
    underprint(f)
    print()
    raw, lines, lines_len = get_lines(txt_dir, f)
    lines_shortened, (_, author_index, end_index) = shortened_lines(f, lines, lines_len, regices, off)
    printlines(lines_shortened)  

---

In [8]:
txt_dir = "theatregratuit-txt"
off = "\t"
if os.path.isdir(txt_dir):
    fnames = [x for x in os.listdir(txt_dir) if ".txt" == os.path.splitext(x)[-1]]
    n_files = len(fnames)

In [10]:
tmp_dir = "tmp1"
if not os.path.isdir(tmp_dir):
    os.mkdir(tmp_dir)

---

In [69]:
def find_characters(fname, lines, lines_len, regices, off):
    char_index = 0
    found_char = False
    for j, l in enumerate(lines):
        if found_char:
            break
        if regex.match(regices["character"], l):
            found_char = True
            k = 1
            while j + k < lines_len and not regex.match(
                regices["blank_line"], lines[j + k]
            ):
                k += 1
            char_index = j + k + 1
    return char_index

In [70]:
def find_author(fname, lines, lines_len, regices, off):
    author_index = 0
    found_author = False
    for j, l in enumerate(lines[:10]):
        if found_author:
            break
        if regex.match(regices["author"], lines[j]):
            found_author = True
            k = 2  # we assume "de\n\nauthor"
            while j + k < lines_len and not regex.match(
                regices["blank_line"], lines[j + k]
            ):
                k += 1
            author_index = j + k + 1
    return author_index

In [71]:
def find_end(fname, lines, lines_len, regices, off):
    end_index = lines_len - 1
    found_end = False
    for j, l in enumerate(reversed(lines[-4:])):
        if found_end:
            break
        if regex.search(regices["fin"], l):
            found_end = True
            end_index = end_index - j - 1
    return end_index

In [9]:
def make_regices():
    return {
        "blank_line": regex.compile("^\s*$"),
        "blank_line_with_rubbish": regex.compile("^[\p{Z}\p{P}]*$"),
        "character": regex.compile(
            "^\s*((les )*pe*rsonna *ge|(les )*acteurs|dramatis personae|entreparleur|biographies|apparences|personrage|persongueules|pépersonâge)",
            regex.IGNORECASE,
        ),
        "characters_block": regex.compile("\n*\s*PERSONNAGES?(.*?)(\n\s*\n)+", regex.DOTALL),
        "author": regex.compile("(^\s*de\s*$|^\(Auteur inconnu\))", regex.IGNORECASE),
        "fin": regex.compile("(fin|rideau|f1n|inachev|manque)", regex.IGNORECASE),
        # search for lines made of full caps + punct/space, as well as some
        # very common words listing characters, and didascalia in ()
        "caps_full": regex.compile("^\s*((\p{Lu}+|seule?|puis|puis tout le monde|moins|et)\s*[.’'<>;,]*\s*)+\s*(\(.*?\))*\s*$"),
        # common annoying starts "M.", "L'"
        "annoying_init": regex.compile("^[\p{P}\p{Z}]*(M\.|\p{Lu}')\p{Z}*"),
        # words (with possible - or '), and space, with punctuation at the end
        "caps_init": regex.compile("^\s*(([\p{Lu}\p{Pc}\p{Pd}1]+\p{Zs}*)+)(\s*[\p{Po}]\s*)"),
        "char_lc_dot_dash": regex.compile("^(\p{Z}*M*[\p{Ll}\p{Z}]+\.)\p{Z}*\p{Pd}+\p{Z}*(.*)$"),
        "caps_word": regex.compile("\p{Lu}{2,}"),
        "trailing_space": regex.compile("\s*$"),
        "trailing_punct": regex.compile("\s*[.,:]*\s+$"),
        "non_breaking_space": regex.compile(" {2,}"), # non-breaking space
        "M." : regex.compile("M.\s"),
        "dot": regex.compile("\s*\.\s*"),
        "final_dot": regex.compile("\s*\.\s*$"),
        "final_comma": regex.compile(",$"),
        # include rare errors like ".-—Blah" or ". -— Blah"
        "dash": regex.compile("[,.](\s*[-–—]+|-[–—])\s*"),
        "dash_and_more": regex.compile("(.*?)[.;:,]*\s*-*[-–—]\s*"),
        "colon": regex.compile("\s*:\s*"),
        "colon_and_more": regex.compile("(.*?)\s*:\s*"),
        "didasc_and_more": regex.compile(".*?(?<!M)[.:]\s*"),
        # "char_no_dot": regex.compile("^\s*([A-Z1'-]+)\s*$"),
        # # using all 3 lengths: -, –, —, bc of inconsistencies
        # "char_dash": regex.compile("^\s*([A-Z1'-]{2,})\.[\s-]+[–—]\s"),
        # "char_comma_dash": regex.compile("^\s*([A-Z1'-]{2,},.*?)\.[\s-]+[–—]\s"),
        # "char_dot_more": regex.compile("^\s*([A-Z1'-]{2,}\.)(.*)$"),
        # "char_no_dot_more": regex.compile("^\s*([A-Z1'-]{2,})(.*)$"),
        # "char_colon": regex.compile("^\s*([A-Z1'-]{2,})\s*:\s*$"),
        # "char_colon_more": regex.compile("^\s*([A-Z1'-]{2,})\s*:\s*(.*)$"),
        "first_scene": regex.compile("(sc[èe]ne (premi[èe]re|I)|premier tableau|I, 1\.)", regex.IGNORECASE)
    }

regices = make_regices()

---

In [7]:
def shortened_lines(fname, lines, lines_len, regices, off):
    char_index = find_characters(fname, lines, lines_len, regices, off)
    if char_index == 0:
        author_index = find_author(fname, lines, lines_len, regices, off)
    else:
        author_index = char_index
    end_index = find_end(fname, lines, lines_len, regices, off)
    # print(fname, char_index, author_index, end_index)
    return lines[author_index:end_index], (char_index, author_index, end_index)

In [190]:
def split_by_regex(fnames, txt_dir, r):
    with_r = []
    without_r = []
    for fname in fnames:
        raw, lines, _ = get_lines(txt_dir, fname)
        found = False
        for l in lines:
            if regex.search(r, l):
                found = True
                break
        if found:
            with_r.append(fname)
        else:
            without_r.append(fname)
    print("with:", len(with_r))
    print("without:", len(without_r))
    return with_r, without_r

---

All files.

In [136]:
fnames_it = iter(fnames)

In [163]:
fname = next(fnames_it)
underprint(fname)
print()
raw, lines, lines_len = get_lines(txt_dir, fname)
lines_shortened, (_, author_index, _) = shortened_lines(fname, lines, lines_len, regices, off)

printlines(lines_shortened)

count-2199212-Les_Paves_de_lours.txt
------------------------------------

Un salon-salle à manger dans un appartement de garçon. Mobilier élégant. Au fond, porte d'entrée — à gauche deuxième plan, une porte — à gauche premier plan, une cheminée. Près de la cheminée un petit guéridon et un canapé tête-à-tête. — A droite, premier plan, une porte. — Deuxième plan, à droite également, un bahut servant de buffet. — A droite, non loin de la porte, une table de travail avec un fauteuil à gauche de la table, faisant face à la porte. — Sur les murs, des tableaux, dont un représente «Léda et son cygne ». — Sur la cheminée, une statuette d'une Diane quelconque, des photographies de femmes, encadrées.
SCENE PREMIERE 
LUCIEN, puis DORA
LUCIEN, à sa table, écrit. — Hélas ! ma chère Dora, il est des circonstances dans la vie, où l'on doit sacrifier son bonheur à son devoir... (Répétant.) à son devoir, hum!... son bonheur à son devoir... voilà des lettres embêtantes à écrire.
DORA, venant de droite. 

---

In [14]:
caps_after_chars = []
no_caps_after_chars = []
for fname in fnames:
    raw, lines, lines_len = get_lines(txt_dir, fname)
    lines_shortened, _ = shortened_lines(fname, lines, lines_len, regices, off)
    for l in lines_shortened:
        if regex.match(regices["blank_line"], l): 
            continue
        if regex.search(regices["caps_word"], l):
            caps_after_chars.append(fname)
            break
        else:
            no_caps_after_chars.append(fname)
            break              

In [15]:
print(len(caps_after_chars))
print(len(no_caps_after_chars))

497
581


---

In [39]:
first_scene, no_first_scene = split_by_regex(fnames, txt_dir, regices["first_scene"])

with: 732
without: 351


In [164]:
# fnames_it = iter(first_scene)
fnames_it = iter(no_first_scene)
index = 0

In [199]:
fname = next(fnames_it)
index += 1
underprint(f"{fname} | {index}")

raw, lines, lines_len = get_lines(txt_dir, fname)
lines_shortened, indices = shortened_lines(fname, lines, lines_len, regices, off)
print("\t"*5 + "char/author/end indices:", indices, "| lines length:", lines_len)
print()

printlines(lines_shortened[:20])
print()
printsep('-')
print()
print_author_split(lines, indices[1])

count-3056514-La_fleur_a_la_bouche.txt | 27
-------------------------------------------
					char/author/end indices: (11, 11, 58) | lines length: 60

On verra au fond les arbres d'un boulevard avec les lampes électriques à travers les feuilles. Des deux côtés, les dernières maisons d'une rue qui aboutit au boulevard. Dans les maisons à gauche, un café sordide avec tables et chaises sur le trottoir. Devant les maisons de droite, un bec de gaz allumé. Au coin de la dernière maison qui sera à l'angle du boulevard, un autre bec de gaz allumé.
Un peu après minuit. On entendra au loin un son de mandoline.
(Au lever du rideau, L’HOMME À LA FLEUR, assis à l'une des tables du café, observera longuement, en silence, le PAISIBLE CLIENT qui, à la table à côté, boira avec une paille un sirop de menthe.)
L'HOMME A LA FLEUR. — Ah ! je me le demandais! Vous donc, homme si paisible, vous avez raté votre train?
LE CLIENT. — Pour une minute ! J'arrive à la gare et je le vois filer sous mon nez !
L'HOMME

---

In [179]:
with_end, without_end = split_by_regex(fnames, txt_dir, regices["fin"])

with: 1083
without: 0


---

In [47]:
# fnames_it = iter(end)
fnames_it = iter(no_end)

In [48]:
fname = next(fnames_it)
print(fname)

raw, lines, lines_len = get_lines(txt_dir, fname)
printlines(lines)
# lines_shortened, author_index = shortened_lines(fname, lines, lines_len, regices, off)

# print_author_split(lines_shortened, author_index)

count-2983302-LAiglon.txt
L’Aiglon
 
de
 
Edmond Rostand
 
PERSONNAGES :
FRANZ, Duc de REICHSTADT
SERAPHIN FLAMBEAU
LE PRINCE DE METTERNICH
L'EMPEREUR FRANZ
LE MARECHAL MARMONT
LE TAILLEUR
FREDERIC DE GENTZ
L'ATTACHE FRANÇAIS
LE CHEVALIER DE PROKESCHOSTEN
TIBURCE DE LORGET
LE COMTE DE DIETRICHSTEIN, précepteur du Duc
LE BARON D'OBENAUS
LE COMTE DE BOMBELLES
LE GENERAL HARTMANN
LE DOCTEUR
LE COMTE DE SEDLINSKY, directeur de la police
UN GARDE-NOBLE
LORD COWLEY, ambassadeur d'Angleterre
THALBERG
FURSTENBERG
MONTENEGRO
UN SERGENT DU REGIMENT DU DUC
LE CAPITAINE FORESTI
UN VIEUX PAYSAN
LE VICOMTE D'OTRANTE
PIONNET
GOUBEAUX
MORCHAIN
BOROKOWSKI
LE VALET DE CHAMBRE DU DUC
L'HUISSIER
UN MONTAGNARD
UN TYROLIEN
UN FERMIER
LE PRELAT
MARIE-LOUISE, duchesse de Parme
LA COMTESSE CAMERATA
THERESE DE LORGET, sœur de Tiburce
L'ARCHIDUCHESSE
FANNY ELSSLER
LA GRANDE-MAITRESSE
PRINCESSE GRAZALCOWITCH
QUELQUES BELLES DAMES DE LA COUR
LADY COWLEY
LES DEMOISELLES D'HONNEUR DE MARIE-LOUISE
UNE VIEILLE PAYSANN